# Long range interaction

Computation of long range phase shifts on the lattice.

This notebook shall identify a local potential with long range components (non-zero effective range but sparse in coordinate or momentum space).
Preferably, the result should be cutoff independent.
For now, $S$-wave results are sufficinet.
The current idea is to choose a potential which mimics chiral LO interactions (contact + one pion exchange) and numerically compute the phase shifts with high precision.

$$\renewcommand{\vec}[1]{\boldsymbol{#1}}$$

---
**Notes:** I first analyzed a separable potential $V(p, p') = - g^*(p) g(p')$ but this is very expensive to compute on the lattice (dense Hamiltonian).


## Numerical compututation of phase shifts

This section provides a numerical extraction of phase shifts.

### Init

Modules:

In [ ]:
import numpy as np
from itertools import product
import matplotlib.pylab as plt

Parameters (in MeV)

In [ ]:
HBARC = 197
mN = 939 /HBARC
mu = mN / 2

Momentum grid points:

In [ ]:
p, wp = np.polynomial.laguerre.laggauss(50)

p = np.linspace(1.e-6, 20, 100)
wp = np.ones(p.size) * (p[1] - p[0])

H0 = np.diag(p**2 / 2 / mu)

### Idea

Compute the $T$-matrix numerically on a partial wave grid $\left| k, \alpha\right\rangle$

\begin{align}
    T_{\alpha'\alpha}\left(k^{\prime}, k ; E\right)
    &=
    V_{\alpha'\alpha}\left(k^{\prime}, k\right)
    +
    \sum_{\alpha''}
    \int d q q^{2} 
    \frac{V_{\alpha'\alpha''}\left(k^{\prime}, q\right) T_{\alpha''\alpha}(q, k ; E)}{E-E_{q}+i \epsilon}
    \\
    &=
    V_{\alpha'\alpha}\left(k^{\prime}, k\right)
    +
    \sum_{\alpha''}\mathcal P \int_{0}^{\infty} d q q^{2} 
        \frac{V_{\alpha'\alpha''}\left(k^{\prime}, q\right) T_{\alpha''\alpha}(q, k ; E)}{E-E_{q}}
    -  i \pi \sum_{\alpha''}\int_{0}^{\infty} d q q^{2} V_{\alpha'\alpha''}\left(k^{\prime}, q\right) T_{\alpha''\alpha}(q, k ; E) \delta(E-E_{q})
\end{align}

where $E_q = q^2 / (2 \mu)$,
$$
    \frac{1}{X+i \epsilon}=\mathcal P \frac{1}{X}-i \pi \delta(X)
    \qquad
    \int d x \,  f(x) \delta(g(x)) = \sum_{x_i} \frac{f(x_i)}{|g'(x_i)|}
$$
was used and $g(x_i) = 0$ are all zeros in the integration domain.
The result is 

$$
    \int_{0}^{\infty} d q q^{2} V_{\alpha'\alpha''}\left(k^{\prime}, q\right) T_{\alpha''\alpha}(q, k ; E) \delta(E-E_{q})
    =
    \frac{\mu}{q} q^2 \, V_{\alpha'\alpha''}\left(k^{\prime}, q\right) T_{\alpha''\alpha}(q, k ; E) \bigg|_{q = \sqrt{2 \mu E}}
$$


The partial wave decomposed matrix element of operators are defined as
$$
    O_{\alpha' \alpha}(k', k)
    =
    \int d \Omega_{k'}  d \Omega_{k}
    Y_{l' m_l'} (\Omega_{k'})
    \left\langle \vec k' , \tilde \alpha' \middle\vert O \middle\vert \vec k, \tilde \alpha \right\rangle
    Y_{l m_l} (\Omega_{k})
$$
with the collection of quantum numbers $\alpha = (\tilde \alpha, l , m_l)$.

More specifically, the angular matrix elements are defined as

\begin{align}
    \left\langle \vec k \middle\vert q l m_l \right\rangle 
    &=
    \frac{\delta(k - q)}{q^2} Y_{l m_l}^*(\Omega_k)
    \, , &
    \mathbb{1}
    &=
    \sum\limits_{l m} \int d q q^2 \left\vert q l m \right\rangle\left\langle q l m \right \vert
\end{align}

In case of $S$-wave only results, this becomes
$$
    O_{0}(k', k) = 
    \frac{1}{4 \pi} 
    \int d \Omega_{k'}  d \Omega_{k}
    \left\langle \vec k' \middle\vert O \middle\vert \vec k \right\rangle
$$

The on-shell equation of the $T$-matrix for $S$-waves only on a numerical grid ($(k_i, w_{k_i})$, $1 \leq i \leq N$) becomes a matrix-vector equation:
\begin{align}
    t_0 &= T_0\left(p_0, p_0, E = \frac{p_0^2}{2 \mu}\right)
    \, , &
    t_i &= T_0\left(k_i, p_0, E = \frac{p_0^2}{2 \mu}\right)
    \, , &
    v_i &= V_0(k_i, p_0)
    \, , &
    v_{i, j} &= V_0(k_i,k_j)
    \, , &
    g_i &= \frac{2 \mu}{p_0^2 - k_i^2}
\end{align}

$$
    t_i = v_i + \sum\limits_{n=1}^N k_n^2 w_{k_n} v_{i n} g_n t_n - i \pi \mu p_0 v_i t_0 
    \, , \qquad  1 \leq i \leq N \, .
$$
One can naturally extend this equation by choosing $k_0 = p_0$ and $k_0 \neq k_i \forall i \neq 0$ such that one finds
\begin{align}
    \sum_{n=0}^N A_{in} t_n &= v_i
    \, , &
    A_{in} &=
    \delta_{in} + 
    \begin{cases}
         - k_n^2 w_{k_n} v_{i n} g_n & n > 0  \\
        i \pi \mu p_0 v_i & n = 0
    \end{cases}
\end{align}

## Potentials

Choose the Yukawa potential 
$$
    V(\vec p, \vec k) 
    = 
    - \frac{g_A^2}{2 F_\pi^2} 
    \frac{m_\pi^2}{(\vec p - \vec k)^2 + m_\pi^2}
    \leftrightarrow
    V(r) 
    = 
   -\frac{m_{\pi }^2 g_A^2}{8 \pi  F_{\pi }^2} \frac{e^{-m_{\pi }r}}{ r}
$$

$g_A = 1.29$, $f_\pi = 132$MeV and $m_\pi = 141$MeV
The $l = 0$ component is given by



$$
\left\langle\mathbf{k}^{\prime}|\widehat{V}| \mathbf{k}\right\rangle=\frac{2}{\pi} \sum_{l, m} V_{l}\left(k^{\prime}, k\right) Y_{l m}^{*}\left(\Omega_{k^{\prime}}\right) Y_{l m}\left(\Omega_{k}\right)
\qquad
V_{l}\left(k^{\prime}, k\right)=\int_{0}^{\infty} r^{2} d r \, j_{l}(k r) V(r) j_{l}\left(k^{\prime} r\right)
$$
adn thus
$$
 V_0(k, k') = \frac{m_{\pi }^2 g_A^2 }{32 \pi 
   F_{\pi }^2 } \frac{1}{k k'}\log \left(1-\frac{4 k k'}{(k+k')^2+m_{\pi }^2}\right)
$$

In [ ]:
def V1(k, mpi=141., gA=1.29, fpi=132.):
    kk = k.reshape(-1, 1)
    return gA**2 * mpi**2/ 32 / np.pi / fpi**2 / kk / k * np.log(1 - 4 * kk * k / ((k+kk)**2 + mpi**2)) 

def V(k):
    return - np.ones([k.size]*2)

## Computing T

In [ ]:
p0 = [(pp + p[n+1])/2 for n, pp in enumerate(p[:-1])][::10]


phase_shifts = []

for pp0 in p0:
    pp = np.append([pp0], p)
    vpp = V(pp)
    Ep = pp**2 / 2 / mu

    A = np.zeros([pp.size]*2, dtype=complex)
    for i, n in product(*[range(pp.size), range(pp.size)]):
        
        if i == n:
            A[i, i] = 1
        
        if n == 0:
            A[i, n] += 1j * np.pi * mu * pp0 * vpp[i, 0]
        else:
            A[i, n] -= p[n-1]**2 * wp[n-1] * vpp[i, n] / (Ep[0] - Ep[n])
            
    t = np.linalg.solve(A , np.diagonal(vpp))
    f = - mu * t * np.pi

    ere = 1 / f + 1j * pp0
    
    phase_shifts.append(ere[0])
    
phase_shifts = np.array(phase_shifts)

In [ ]:
fig, ax = plt.subplots()

ax.plot(pp[1:], A[0, 1:].real, ".")
ax.axvline(pp[0])

ax.set_ylim(-10, 10)

plt.show()

In [ ]:
np.sum(A.real, axis=1)

In [ ]:
fig, ax = plt.subplots()

ax.plot(pp[1:], A[0, 1:].imag, ".")
ax.axvline(pp[0])

ax.set_ylim(-10, 10)

plt.show()

In [ ]:
plt.plot(p0, phase_shifts.imag, ".")

In [ ]:
(1 / t).real

In [ ]:
p

In [ ]:
p0

In [ ]:
len(1)